In [ ]:
#default_exp config
%load_ext autoreload
from nbdev.showdoc import show_doc

# Configuration data and basic functions
> Basic functions and configuration stuff

## Time conversion

- MET: mission elapsed time
- MJD: modified Julian date (days)
- UTC: ISO time

In [ ]:
#export
from astropy.time import Time
from astropy.coordinates import SkyCoord
from dataclasses import dataclass
from pathlib import Path
from typing import Tuple
import os, sys
import numpy as np

In [ ]:
#export

day = 24*3600.
first_data=54683

def MJD(met):
    "convert MET to MJD"
    #mission_start = Time('2001-01-01T00:00:00', scale='utc').mjd
    # From a FT2 file header
    # MJDREFI =               51910. / Integer part of MJD corresponding to SC clock S
    # MJDREFF =  0.00074287037037037 / Fractional part of MJD corresponding to SC cloc
    mission_start = 51910.00074287037
    return (mission_start + met/day  )

def UTC(mjd):
    " convert MJD value to ISO date string"
    t=Time(mjd, format='mjd')
    t.format='iso'; t.out_subfmt='date_hm'
    return t.value

In [ ]:
assert UTC(MJD(0))=='2001-01-01 00:01'

In [ ]:
#export
import pickle

class Cache(dict):
    """
    Manage a file cache

    - `path` -- string or `filepath` object
    - `clear` -- set True to clear the cache

    """
    
    def __init__(self, path, clear:bool=False):


        self.path = Path(path) if path else None
        if not self.path: return
        self.path.mkdir(exist_ok=True)
        assert self.path.exists()
        self.index_file = self.path/'index.pkl'

        if self.path.exists():  
            if clear: 
                print('Clearing cache!')
                self.clear()
            else:
                self._load_index()
        else:
            self.path.mkdir(exist_ok=True)        

    def _dump_index(self):
        with open(self.index_file, 'wb') as file:
            pickle.dump(self, file)
        
    def _load_index(self):
        if not self.index_file.exists(): 
            self._dump_index()
            return
        with open(self.index_file, 'rb') as file:
            self.update(pickle.load(file))
        
    def add(self, key, object, exist_ok=False):
        if not self.path: return
        if key  in self:
            if not exist_ok:
                print(f'Warning: cached object for key "{key}" exists', file=sys.stderr)
            filename = self[key]
        else:
            filename = self.path/f'cache_file_{hex(key.__hash__())[3:]}.pkl'
            self[key] = filename
            self._dump_index() 
 
        with open(filename, 'wb') as file:
            pickle.dump(object, file )

        
    def get(self, key):
        if key not in self:
            return None
        filename = self[key]
        with open(filename, 'rb') as file:
            ret = pickle.load(file)
        return ret
    
    def clear(self):
        if not self.path: return
        for f in self.path.iterdir():
            if f.is_file: 
                f.unlink()
        super().clear()

        self._dump_index()
        
    def remove(self, key):
        """remove entry and associated file"""
        if not self.path: return
        if key not in self:
            print(f'Cache: key {key} not found', file=sys.stderr)
            return
        filename = self[key]
        filename.unlink()
        super().pop(key)
        self._dump_index()
        
                
    def __call__(self, key, func, *pars, description='', overwrite=False, **kwargs,
                ):
        """
        One-line usage interface for cache use
        
        - `key` -- key to use, usually a string. Must be hashable <br>
            If none, ignore cache and return the function evaluation
        - `func` -- user function that will return an object that can be pickled
        - `pars`, `kwargs` -- pass to `func`
        - `description` -- optional string that will be printed

        
        cache = Cache(path)
        result = cache(key, function, *pars, **kwargs)
        """
        
        if key is None:
            return func(*pars, **kwargs)
        
        ret = self.get(key)
        if description:
            print(f'{description}: {"Saving to" if key not in self else "Restoring from"} cache with key "{key}"')

        if ret is None or overwrite: 
            ret = func(*pars, **kwargs)
            self.add(key, ret)
        return ret
             
    def __str__(self):
        import datetime
        if not self.path: return 'Cache not enabled'
        s = f'Cache contents\n {"key":20}   {"size":>10}  {"time":20} {"name"}, in folder {self.path}\n'
        for name, value in self.items():
            if name is None: continue
            stat = value.stat()
            size = stat.st_size
            mtime= str(datetime.datetime.fromtimestamp(stat.st_mtime))[:16]
            s += f'  {name:20s}  {size:10}  {mtime:20} {value.name}\n'
        return s

In [ ]:
#collapse_hide
show_doc(Cache)
show_doc(Cache.__call__)
def cache_test(path):
    c = Cache(path, clear=True)

    # simmple interface
    c.add('one', 'one');
    c.add('two', 'two')
    c.add('two', 'two') # getnerates warning
    if path is not None:
        assert c.get('two') == 'two'

    # test function interface
    func = lambda x:f'value: {x}'
    
    r1 = c('four',  func,  4, description='Test')
    r2 = c('four',  func,  5,  description='Test') #should not get called
    assert c.path is None or r1==r2, f'{r1}, {r2}'
    
    # remaving an entry
    print(f'Before remove:\n{c}')
    assert 'four' in c
    c.remove('four')
    assert 'four' not in c
   
    
    c.clear()
cache_test('/tmp/test_cache')
# disabled should ignore
#cache_test(None)

<h2 id="Cache" class="doc_header"><code>class</code> <code>Cache</code><a href="" class="source_link" style="float:right">[source]</a></h2>

> <code>Cache</code>(**`path`**, **`clear`**:`bool`=*`False`*) :: `dict`

Manage a file cache

- `path` -- string or `filepath` object
- `clear` -- set True to clear the cache

<h4 id="Cache.__call__" class="doc_header"><code>Cache.__call__</code><a href="__main__.py#L86" class="source_link" style="float:right">[source]</a></h4>

> <code>Cache.__call__</code>(**`key`**, **`func`**, **\*`pars`**, **`description`**=*`''`*, **`overwrite`**=*`False`*, **\*\*`kwargs`**)

One-line usage interface for cache use

- `key` -- key to use, usually a string. Must be hashable <br>
    If none, ignore cache and return the function evaluation
- `func` -- user function that will return an object that can be pickled
- `pars`, `kwargs` -- pass to `func`
- `description` -- optional string that will be printed


cache = Cache(path)
result = cache(key, function, *pars, **kwargs)

Clearing cache!
Test: Saving to cache with key "four"
Test: Restoring from cache with key "four"
Before remove:
Cache contents
 key                          size  time                 name, in folder /tmp/test_cache
  one                           13  2020-12-18 10:08     cache_file_ff9a98a1b6f7456.pkl
  two                           13  2020-12-18 10:08     cache_file_a24b9e6f3e5af9c.pkl
  four                          18  2020-12-18 10:08     cache_file_3a7ac979522ff17a.pkl



## Configuration data classes


In [ ]:
#export
@dataclass
class Files:
    """ paths to the various files that we need"""

    data:str = '$HOME/data'
    ft2: str = '$HOME/work/lat-data/ft2'
    gti: str = '$HOME/work/lat-data/binned/'
    aeff:str = '$HOME/work/lat-data/aeff'
    weights: str = '$HOME/onedrive/fermi/weight_files'
    cachepath: str = '/tmp/lc_cache'

    # expand -- not implemented in Path
    def __post_init__(self):
        d = self.__dict__
        for name, value in d.items():
            d[name] = Path(os.path.expandvars(value))
#         self.cachepath.mkdir(exist_ok=True)
#         self.cache = Cache(self.cachepath, clear=False)
        
    @property
    def valid(self):
        """assume all files ok if aeff"""
        return self.aeff.exists()
    
    def __repr__(self):
        s = 'File paths for light curves\n'
        for name, value in self.__dict__.items():
            s += f'  {name:10s} : {value}\n'
        return s

In [ ]:
Files()

File paths for light curves
  data       : /home/burnett/data
  ft2        : /home/burnett/work/lat-data/ft2
  gti        : /home/burnett/work/lat-data/binned
  aeff       : /home/burnett/work/lat-data/aeff
  weights    : /home/burnett/onedrive/fermi/weight_files
  cachepath  : /tmp/lc_cache

In [ ]:
#export
@dataclass
class Config:
    """Default light curve configuration parameters"""
    verbose : int = 3
    files :'' =  None

    # photon selection
    mjd_range : Tuple=None
    radius: float = 5
    cos_theta_max:float=0.4
    z_max : float=100

    # binning
    energy_edges = np.logspace(2,6,17)
    time_interval: int = 1

    # healpix data representation used by data
    nside : int=1024
    nest: bool=True

    # exposure calculation
    bins_per_decade: int=4
    base_spectrum: str='lambda E: (E/1000)**-2.1'
    energy_range: Tuple = (100.,1e6)
        
    # analysis
    likelihood_rep: str='poisson'
        
    def __post_init__(self):
        if self.files is None: self.files=Files()
        chpath = self.files.cachepath
        chpath.mkdir(exist_ok=True)
        self.cache = Cache(chpath, clear=False)
        
 
    @property
    def valid(self):
        return self.files.valid
    
    def __str__(self):
        s = 'Configuration parameters \n'
        for name, value in self.__dict__.items():
            if name=='files': continue
            s += f'  {name:15s} : {value}\n'
        return s

    def __repr__(self): return str(self)

In [ ]:
config = Config()
print(config.cache)

Cache contents
 key                          size  time                 name, in folder /tmp/lc_cache
  gti                      1018319  2020-12-17 15:17     cache_file_26e0b050cac876b4.pkl
  exposure_Geminga        86263776  2020-12-17 15:47     cache_file_3d18a74e668c437.pkl
  lightcurve_Geminga      12193571  2020-12-17 15:50     cache_file_5f8c1fe2233119ff.pkl
  photons_3C 279           3659049  2020-12-17 15:50     cache_file_005a8b245230afe.pkl
  exposure_3C 279         78040992  2020-12-17 15:51     cache_file_86f97125c87ecd8.pkl
  lightcurve_3C 279        4687729  2020-12-17 15:51     cache_file_509522c898fe52.pkl
  photons_Geminga         22334705  2020-12-18 06:39     cache_file_52b89da960a98f6.pkl
  cells_Geminga            1500794  2020-12-18 08:48     cache_file_6f2eb7e7fa4947f8.pkl
                           1500794  2020-12-18 08:51     cache_file_.pkl
  lightfcurve_Geminga     12193634  2020-12-18 09:06     cache_file_f721860d03c0aa7.pkl
  cells_3C 279              557

In [ ]:
#export                
class PointSource():
    """Manage the position and name of a point source
    """
    def __init__(self, name, position=None):
        """position: (l,b) tuple or None. if None, expect to be found by lookup
        """
        self.name=name
        if position is None:
            skycoord = SkyCoord.from_name(name)
            gal = skycoord.galactic
            self.l,self.b = (gal.l.value, gal.b.value)
        else:
            self.l,self.b = position
            skycoord = SkyCoord(self.l,self.b, unit='deg', frame='galactic')
        self.skycoord = skycoord
    def __str__(self):
        return f'Source "{self.name}" at: (l,b)=({self.l:.3f},{self.b:.3f})'
    def __repr__(self): return str(self)

    @property
    def ra(self):
        sk = self.skycoord.transform_to('fk5')
        return sk.ra.value
    @property
    def dec(self):
        sk = self.skycoord.transform_to('fk5')
        return sk.dec.value

    @property
    def filename(self):
        """Modified name for file system"""
        return self.name.replace(' ', '_').replace('+','p')

    @classmethod
    def fk5(cls, name, position):
        """position: (ra,dec) tuple """
        ra,dec = position
        sk = SkyCoord(ra, dec, unit='deg',  frame='fk5').transform_to('galactic')
        return cls(name, (sk.l.value, sk.b.value))

In [ ]:
show_doc(PointSource.fk5)

<h4 id="PointSource.fk5" class="doc_header"><code>PointSource.fk5</code><a href="__main__.py#L35" class="source_link" style="float:right">[source]</a></h4>

> <code>PointSource.fk5</code>(**`name`**, **`position`**)

position: (ra,dec) tuple 

In [ ]:
for s, expect in [( PointSource('Geminga'),             'Source "Geminga" at: (l,b)=(195.134,4.266)'),
                  ( PointSource('gal_source', (0,0)),   'Source "gal_source" at: (l,b)=(0.000,0.000)', ),
                  ( PointSource.fk5('fk5_source',(0,0)),'Source "fk5_source" at: (l,b)=(96.337,-60.189)',)
                   ]:    
    assert str(s)==expect, f'expected {expect}, got {str(s)}'


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()
!date

Converted 00_config.ipynb.
Converted 01_effective_area.ipynb.
Converted 02_load_gti.ipynb.
Converted 03_exposure.ipynb.
Converted 04_photon_data.ipynb.
Converted 05_weights.ipynb.
Converted 07_cells.ipynb.
Converted 08_simulation.ipynb.
Converted 09_poisson.ipynb.
Converted 10_loglike.ipynb.
Converted 11_lightcurve.ipynb.
Converted 12_roadmap.ipynb.
Converted 13_kerr_comparison.ipynb.
Converted 14_bayesian.ipynb.
Converted index.ipynb.
Fri Dec 18 10:08:08 PST 2020
